# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 16 2023 10:14AM,255050,15,VT80182798,"Virtus Pharmaceuticals, LLC",Released
1,Jan 16 2023 10:14AM,255050,15,VT80182799,"Virtus Pharmaceuticals, LLC",Released
2,Jan 16 2023 10:14AM,255050,15,VT80182800,"Virtus Pharmaceuticals, LLC",Released
3,Jan 16 2023 10:14AM,255050,15,VT80182801,"Virtus Pharmaceuticals, LLC",Released
4,Jan 16 2023 10:14AM,255050,15,VT80182802,"Virtus Pharmaceuticals, LLC",Released
5,Jan 16 2023 10:14AM,255050,15,VT80182838,"Virtus Pharmaceuticals, LLC",Released
6,Jan 16 2023 10:14AM,255050,15,VT80182839,"Virtus Pharmaceuticals, LLC",Released
7,Jan 16 2023 10:14AM,255050,15,VT80182840,"Virtus Pharmaceuticals, LLC",Released
8,Jan 16 2023 10:14AM,255050,15,VT80182841,"Virtus Pharmaceuticals, LLC",Released
9,Jan 16 2023 10:14AM,255050,15,VT80182842,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,255072,Released,9
19,255073,Released,36
20,255074,Released,34
21,255076,Released,46
22,255077,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
255072,NaN,9.0
255073,NaN,36.0
255074,NaN,34.0
255076,NaN,46.0
255077,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
255006,21.0,31.0
255049,0.0,42.0
255050,0.0,16.0
255053,0.0,1.0
255054,2.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
255006,21,31
255049,0,42
255050,0,16
255053,0,1
255054,2,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,255006,21,31
1,255049,0,42
2,255050,0,16
3,255053,0,1
4,255054,2,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,255006,21,31
1,255049,,42
2,255050,,16
3,255053,,1
4,255054,2,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 16 2023 10:14AM,255050,15,"Virtus Pharmaceuticals, LLC"
16,Jan 16 2023 9:43AM,255077,10,ISDIN Corporation
17,Jan 16 2023 9:40AM,255076,10,ISDIN Corporation
63,Jan 16 2023 9:39AM,255074,10,ISDIN Corporation
97,Jan 16 2023 9:39AM,255073,10,ISDIN Corporation
133,Jan 16 2023 9:34AM,255072,10,ISDIN Corporation
142,Jan 16 2023 9:31AM,255066,10,ISDIN Corporation
195,Jan 16 2023 9:30AM,255049,15,"Brookfield Pharmaceuticals, LLC"
237,Jan 16 2023 9:30AM,255064,10,ISDIN Corporation
278,Jan 16 2023 9:29AM,255006,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 16 2023 10:14AM,255050,15,"Virtus Pharmaceuticals, LLC",,16
1,Jan 16 2023 9:43AM,255077,10,ISDIN Corporation,,1
2,Jan 16 2023 9:40AM,255076,10,ISDIN Corporation,,46
3,Jan 16 2023 9:39AM,255074,10,ISDIN Corporation,,34
4,Jan 16 2023 9:39AM,255073,10,ISDIN Corporation,,36
5,Jan 16 2023 9:34AM,255072,10,ISDIN Corporation,,9
6,Jan 16 2023 9:31AM,255066,10,ISDIN Corporation,,53
7,Jan 16 2023 9:30AM,255049,15,"Brookfield Pharmaceuticals, LLC",,42
8,Jan 16 2023 9:30AM,255064,10,ISDIN Corporation,,41
9,Jan 16 2023 9:29AM,255006,15,"Person & Covey, Inc.",21,31


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 10:14AM,255050,15,"Virtus Pharmaceuticals, LLC",16,
1,Jan 16 2023 9:43AM,255077,10,ISDIN Corporation,1,
2,Jan 16 2023 9:40AM,255076,10,ISDIN Corporation,46,
3,Jan 16 2023 9:39AM,255074,10,ISDIN Corporation,34,
4,Jan 16 2023 9:39AM,255073,10,ISDIN Corporation,36,
5,Jan 16 2023 9:34AM,255072,10,ISDIN Corporation,9,
6,Jan 16 2023 9:31AM,255066,10,ISDIN Corporation,53,
7,Jan 16 2023 9:30AM,255049,15,"Brookfield Pharmaceuticals, LLC",42,
8,Jan 16 2023 9:30AM,255064,10,ISDIN Corporation,41,
9,Jan 16 2023 9:29AM,255006,15,"Person & Covey, Inc.",31,21


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 10:14AM,255050,15,"Virtus Pharmaceuticals, LLC",16,
1,Jan 16 2023 9:43AM,255077,10,ISDIN Corporation,1,
2,Jan 16 2023 9:40AM,255076,10,ISDIN Corporation,46,
3,Jan 16 2023 9:39AM,255074,10,ISDIN Corporation,34,
4,Jan 16 2023 9:39AM,255073,10,ISDIN Corporation,36,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 10:14AM,255050,15,"Virtus Pharmaceuticals, LLC",16.0,NaN
1,Jan 16 2023 9:43AM,255077,10,ISDIN Corporation,1.0,NaN
2,Jan 16 2023 9:40AM,255076,10,ISDIN Corporation,46.0,NaN
3,Jan 16 2023 9:39AM,255074,10,ISDIN Corporation,34.0,NaN
4,Jan 16 2023 9:39AM,255073,10,ISDIN Corporation,36.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 10:14AM,255050,15,"Virtus Pharmaceuticals, LLC",16.0,0.0
1,Jan 16 2023 9:43AM,255077,10,ISDIN Corporation,1.0,0.0
2,Jan 16 2023 9:40AM,255076,10,ISDIN Corporation,46.0,0.0
3,Jan 16 2023 9:39AM,255074,10,ISDIN Corporation,34.0,0.0
4,Jan 16 2023 9:39AM,255073,10,ISDIN Corporation,36.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3060821,239.0,18.0
12,765178,21.0,1.0
15,765105,89.0,21.0
16,255061,1.0,0.0
20,255070,1.0,0.0
22,255053,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3060821,239.0,18.0
1,12,765178,21.0,1.0
2,15,765105,89.0,21.0
3,16,255061,1.0,0.0
4,20,255070,1.0,0.0
5,22,255053,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,239.0,18.0
1,12,21.0,1.0
2,15,89.0,21.0
3,16,1.0,0.0
4,20,1.0,0.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,239.0
1,12,Released,21.0
2,15,Released,89.0
3,16,Released,1.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,20,22
Status,,,,,,
Executing,18.0,1.0,21.0,0.0,0.0,0.0
Released,239.0,21.0,89.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,20,22
0,Executing,18.0,1.0,21.0,0.0,0.0,0.0
1,Released,239.0,21.0,89.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,20,22
0,Executing,18.0,1.0,21.0,0.0,0.0,0.0
1,Released,239.0,21.0,89.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()